# Features usados


 - Factor c / v c / s
 - First and last date month 5
 - New Returning , device type (smartphone or computer)
 -  <b> Score: 0.73360 </b>

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features

import matplotlib #collection of functions for scientific and publication-ready visualization

import numpy as np #foundational package for scientific computing

import scipy as sp #collection of functions for scientific computing and advance mathematics

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook

import sklearn #collection of machine learning algorithms

#misc libraries
import random
import time
import datetime as dt

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

import featuretools as ft
from sklearn.feature_extraction.text import CountVectorizer


-------------------------


In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import xgboost as xgb

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8



In [3]:
data_raw = pd.read_csv('events_up_to_01062018.csv')
data_val = pd.read_csv('labels_training_set.csv')

In [4]:
df_labels = data_val.copy(deep=True) 
df = data_raw.copy(deep=True)

In [5]:
pd.set_option('display.max_columns', 23)

In [6]:
df.describe(include= 'all')

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1.320530e+06,1321513,1320530,1320530,1320530,505949,113763,11201,191286,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,NaN,208,5,8,63,52267,10964,14,23,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,NaN,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",Iphone,CustomerService,google,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,NaN,107262,547617,442096,314925,2606,2577,5239,123354,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953
mean,NaN,NaN,NaN,NaN,6.899178e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,4.028042e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,7.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.929000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,7.057000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.001400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
persons = (df.drop_duplicates('person'))['person'].to_frame()
persons_to_train = df_labels['person'].to_frame()
persons_to_predict = persons.loc[~persons['person'].isin(persons_to_train['person'])]


print(persons_to_train.shape)
print(persons_to_predict.shape)


(19414, 1)
(19415, 1)


In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month'] = df['timestamp'].dt.month
df_month = df.loc[df['month'] == 5]

In [9]:
df_month.shape

(1713920, 24)

# Converted

In [10]:
df_modelos_conversion = df_month.loc[df_month['event']=='conversion']
df_modelos_conversion = df_modelos_conversion['model'].value_counts().to_frame()

In [11]:
df_modelos_conversion['modelo'] =df_modelos_conversion.index
df_modelos_conversion = df_modelos_conversion.reset_index(drop=True)
df_modelos_conversion.columns = ['cantidad', 'modelo']

In [12]:
df_modelos_conversion['modelo']= df_modelos_conversion['modelo'].str.lower()

In [13]:
df_modelos_conversion.head()

,cantidad,modelo
0,371,iphone 5s
1,355,samsung galaxy j5
2,287,iphone 6
3,274,iphone 6s
4,221,motorola moto g2 3g dual


# Viewed

In [14]:
df_modelos_viewed = df_month.loc[df_month['event']=='viewed product']
df_modelos_viewed = df_modelos_viewed['model'].value_counts().to_frame()

In [15]:
df_modelos_viewed['modelo'] =df_modelos_viewed.index
df_modelos_viewed = df_modelos_viewed.reset_index(drop=True)
df_modelos_viewed.columns = ['cantidad', 'modelo']

In [16]:
df_modelos_viewed['modelo']= df_modelos_viewed['modelo'].str.lower()

In [17]:
df_modelos_viewed.head()

,cantidad,modelo
0,72321,iphone 6s
1,69669,iphone 6
2,64615,iphone 5s
3,40771,iphone 7
4,34401,samsung galaxy s7 edge


# Searched


In [18]:
df_modelos_searched = df_month.loc[df_month['event']=='searched products']
df_modelos_searched = df_modelos_searched['search_term'].value_counts().to_frame()

In [19]:
df_modelos_searched['term'] =df_modelos_searched.index
df_modelos_searched = df_modelos_searched.reset_index(drop=True)
df_modelos_searched.columns = ['cantidad', 'modelo']

In [20]:
df_modelos_searched['modelo']= df_modelos_searched['modelo'].str.lower()

In [21]:
df_modelos_searched = df_modelos_searched.groupby('modelo').agg({'cantidad':'sum'}).reset_index().sort_values(by='cantidad',ascending = False)

In [22]:
df_modelos_searched = df_modelos_searched.reset_index(drop= True)
df_modelos_searched.head()

,modelo,cantidad
0,iphone 6,5056
1,iphone 6s,4448
2,iphone,3701
3,iphone 7,2105
4,iphone 5s,2058


# Factor de CONVERSION / VISTO


In [23]:
df_factor_cv = df_modelos_conversion['modelo'].to_frame()

In [24]:
df_factor_cv['relacion c/v'] = df_modelos_conversion['cantidad'] / df_modelos_viewed ['cantidad']

In [25]:
df_factor_cv.sort_values(by='relacion c/v' , ascending=False).head()

,modelo,relacion c/v
48,samsung galaxy j3,0.011026
47,lg k10 tv,0.010931
46,samsung galaxy j1 2016,0.010819
44,motorola moto x play 4g dual,0.010299
42,samsung galaxy a5 2016,0.010213


In [26]:
df_factor_cv.index = df_factor_cv['modelo']
df_factor_cv = df_factor_cv.drop('modelo', axis = 1)
df_factor_cv.head()

,relacion c/v
modelo,
iphone 5s,0.005130
samsung galaxy j5,0.005096
iphone 6,0.004442
iphone 6s,0.006720
motorola moto g2 3g dual,0.006424


# Factor de CONVERSION / BUSCADO 

- Cuidado con este ya que al tener que buscar muy exacto genera ruido por ejemplo el  <b>motorola moto g2 3g dual <\b>

In [27]:
df_factor_cs = df_modelos_conversion.merge(df_modelos_searched, on='modelo', how='left')
df_factor_cs.head()

,cantidad_x,modelo,cantidad_y
0,371,iphone 5s,2058.0
1,355,samsung galaxy j5,63.0
2,287,iphone 6,5056.0
3,274,iphone 6s,4448.0
4,221,motorola moto g2 3g dual,NaN


In [28]:
df_factor_cs['relacion c/s'] = df_factor_cs['cantidad_x'] / df_factor_cs ['cantidad_y']

In [29]:
df_factor_cs = df_factor_cs.drop(['cantidad_x','cantidad_y'],axis=1)

In [30]:
df_factor_cs.head()

,modelo,relacion c/s
0,iphone 5s,0.180272
1,samsung galaxy j5,5.634921
2,iphone 6,0.056764
3,iphone 6s,0.061601
4,motorola moto g2 3g dual,NaN


In [31]:
df_factor_cs.index = df_factor_cs['modelo']
df_factor_cs = df_factor_cs.drop('modelo', axis = 1)
df_factor_cv.head()

,relacion c/v
modelo,
iphone 5s,0.005130
samsung galaxy j5,0.005096
iphone 6,0.004442
iphone 6s,0.006720
motorola moto g2 3g dual,0.006424


# Uso para mis features

## Viewed product

In [80]:
df_conversion = df_month.loc[df_month['event'] == 'conversion']
models_converted = (df_conversion['model'].value_counts())
df_top_models =models_converted.index

df_top_models = pd.Series(df_top_models)
models_converted = df_top_models.values
models_converted = np.array([x.lower() if isinstance(x, str) else x for x in models_converted])
models_converted = list(models_converted)

In [81]:
models_converted

['iphone 5s',
 'samsung galaxy j5',
 'iphone 6',
 'iphone 6s',
 'motorola moto g2 3g dual',
 'samsung galaxy j7 prime',
 'samsung galaxy s7',
 'motorola moto g4 plus',
 'samsung galaxy s6 flat',
 'samsung galaxy s8',
 'samsung galaxy s7 edge',
 'iphone 7 plus',
 'samsung galaxy a5 2017',
 'iphone 4s',
 'iphone 6 plus',
 'samsung galaxy s6 edge',
 'iphone 5c',
 'iphone 7',
 'samsung galaxy j7',
 'samsung galaxy j2 prime tv',
 'samsung galaxy gran prime duos tv',
 'samsung galaxy win duos',
 'lenovo vibe k5',
 'samsung galaxy core plus duos tv',
 'samsung galaxy s5',
 'motorola moto g5 plus',
 'motorola moto g5 ',
 'samsung galaxy j2 4g duos tv',
 'iphone se',
 'samsung galaxy a5',
 'samsung galaxy s5 duos',
 'samsung galaxy j5 prime',
 'samsung galaxy j1 mini',
 'iphone 6s plus',
 'samsung galaxy j7 2016 metal',
 'samsung galaxy a7 2017',
 'iphone 4g',
 'motorola moto g3 hdtv',
 'motorola moto g3 4g',
 'samsung galaxy s5 mini duos',
 'samsung galaxy s8 plus',
 'iphone 5',
 'samsung gala

In [82]:
df_model_viewed = df.loc[df['event'] == 'viewed product']
df_model_viewed = df_model_viewed.groupby('person')['model'].apply(list).reset_index()
df_model_viewed.head()

,person,model
0,00091926,"[iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola..."
1,00091a7a,"[iPhone SE, iPhone 6, iPhone 6S]"
2,000ba417,"[Samsung Galaxy A3 2016, Samsung Galaxy Gran P..."
3,000c79fe,"[iPhone 7, iPhone 7, iPhone 7]"
4,000e4d9e,"[Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fl..."


In [83]:
df_model_viewed['model'] = df_model_viewed['model'].apply(lambda x: ', '.join(map(str, x)))
df_model_viewed = df_model_viewed.fillna('')
df_model_viewed.head()

,person,model
0,00091926,"iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola ..."
1,00091a7a,"iPhone SE, iPhone 6, iPhone 6S"
2,000ba417,"Samsung Galaxy A3 2016, Samsung Galaxy Gran Pr..."
3,000c79fe,"iPhone 7, iPhone 7, iPhone 7"
4,000e4d9e,"Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fla..."


In [84]:
word_vectorizer = CountVectorizer(vocabulary = models_converted, tokenizer=lambda x: x.split(', '))

tf_mat = word_vectorizer.fit_transform(df_model_viewed['model'])

tf_array = tf_mat.toarray()
tf_array.shape

(37130, 144)

In [85]:
model_names = word_vectorizer.get_feature_names()

In [86]:
df_count = pd.DataFrame(tf_array, columns=model_names)
df_count.shape

(37130, 144)

In [87]:
df_count.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,lg x screen,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g
0,0,1,5,94,0,7,1,0,15,5,9,...,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
2,1,11,0,0,0,0,0,0,1,0,0,...,0,0,4,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
4,7,1,0,1,0,9,22,0,139,9,2,...,0,0,0,0,0,0,0,0,0,0,0


In [88]:
df_count = df_count.transpose()

In [89]:
df_count_cv = df_count.mul(df_factor_cv.reindex(df_count.index)['relacion c/v'], axis=0)

In [90]:
df_count_cv = df_count_cv.transpose()

In [91]:
df_count_cv.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,lg x screen,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g
0,0.000000,0.005096,0.022208,0.631724,0.0,0.042270,0.00568,0.0,0.087901,0.020944,0.040710,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.000000,0.004442,0.006720,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.005130,0.056051,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.005860,0.000000,0.000000,...,0.0,0.0,0.016393,0.0,0.0,0.0,0.0,0.0,0.005525,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.035909,0.005096,0.000000,0.006720,0.0,0.054347,0.12497,0.0,0.814545,0.037699,0.009047,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [92]:
df_count_cv['total_cv_factor'] = df_count_cv.iloc[:, 1:].sum(1)
df_count_cv.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g,total_cv_factor
0,0.000000,0.005096,0.022208,0.631724,0.0,0.042270,0.00568,0.0,0.087901,0.020944,0.040710,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.292824
1,0.000000,0.000000,0.004442,0.006720,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.018119
2,0.005130,0.056051,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.005860,0.000000,0.000000,...,0.0,0.016393,0.0,0.0,0.0,0.0,0.0,0.005525,0.0,0.0,1.002310
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.014998
4,0.035909,0.005096,0.000000,0.006720,0.0,0.054347,0.12497,0.0,0.814545,0.037699,0.009047,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.063399


In [93]:
df_count_cv['person'] = df_model_viewed['person']

In [94]:
df_factorcv_by_model = df_count_cv['total_cv_factor'].to_frame()
df_factorcv_by_model.sort_values(by='total_cv_factor',ascending=False).head()

,total_cv_factor
15478,11.666230
990,11.092178
20729,10.074998
16084,9.834870
15045,9.615904


In [95]:
df_count_cv.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g,total_cv_factor,person
0,0.000000,0.005096,0.022208,0.631724,0.0,0.042270,0.00568,0.0,0.087901,0.020944,0.040710,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.292824,00091926
1,0.000000,0.000000,0.004442,0.006720,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.018119,00091a7a
2,0.005130,0.056051,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.005860,0.000000,0.000000,...,0.016393,0.0,0.0,0.0,0.0,0.0,0.005525,0.0,0.0,1.002310,000ba417
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.014998,000c79fe
4,0.035909,0.005096,0.000000,0.006720,0.0,0.054347,0.12497,0.0,0.814545,0.037699,0.009047,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.063399,000e4d9e


In [96]:
df_count_cv['person'] = df_model_viewed['person']
df_count_cv.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g,total_cv_factor,person
0,0.000000,0.005096,0.022208,0.631724,0.0,0.042270,0.00568,0.0,0.087901,0.020944,0.040710,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.292824,00091926
1,0.000000,0.000000,0.004442,0.006720,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.018119,00091a7a
2,0.005130,0.056051,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.005860,0.000000,0.000000,...,0.016393,0.0,0.0,0.0,0.0,0.0,0.005525,0.0,0.0,1.002310,000ba417
3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.014998,000c79fe
4,0.035909,0.005096,0.000000,0.006720,0.0,0.054347,0.12497,0.0,0.814545,0.037699,0.009047,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.063399,000e4d9e


In [97]:
df_count_cv['total_cv_factor'] = df_count_cv.iloc[:, 0:151].sum(1)

In [98]:
df_count_cv = df_count_cv [['person' , 'total_cv_factor']]
df_count_cv.head()

,person,total_cv_factor
0,00091926,4.585649
1,00091a7a,0.036239
2,000ba417,2.009749
3,000c79fe,0.029996
4,000e4d9e,4.162707


## Searched product

In [99]:
df_model_viewed = df_month.loc[df_month['event'] == 'searched products']
df_model_viewed = df_model_viewed.groupby('person')['search_term'].apply(list).reset_index()
df_model_viewed.head()

,person,search_term
0,000c79fe,"[Iphone 7, Galaxy a8, Iphone 7, nan, Galaxy s8..."
1,000e619d,"[samsung rosa, sansung j7, sansung j7, sansung..."
2,001001be,"[IPhone 6, 5s, IPhone 6, IPhone 6, 5s, IPho..."
3,001802e4,"[Aiphone 6s, nan, nan, Aiphone 6s]"
4,0019e639,[ON 7]


In [100]:
df_model_viewed['search_term'] = df_model_viewed['search_term'].apply(lambda x: ', '.join(map(str, x)))
df_model_viewed = df_model_viewed.fillna('')
df_model_viewed.head()

,person,search_term
0,000c79fe,"Iphone 7, Galaxy a8, Iphone 7, nan, Galaxy s8,..."
1,000e619d,"samsung rosa, sansung j7, sansung j7, sansung ..."
2,001001be,"IPhone 6, 5s, IPhone 6, IPhone 6, 5s, IPhon..."
3,001802e4,"Aiphone 6s, nan, nan, Aiphone 6s"
4,0019e639,ON 7


In [101]:
word_vectorizer = CountVectorizer(vocabulary = models_converted, tokenizer=lambda x: x.split(', '))

tf_mat = word_vectorizer.fit_transform(df_model_viewed['search_term'])

tf_array = tf_mat.toarray()
tf_array.shape

(11310, 144)

In [102]:
model_names = word_vectorizer.get_feature_names()

In [103]:
df_count = pd.DataFrame(tf_array, columns=model_names)
df_count.shape

(11310, 144)

In [104]:
df_count.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,lg x screen,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g
0,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0


In [125]:
df_count = df_count.transpose()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,lg x screen,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g
0,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0


In [106]:
df_count_cs = df_count.mul(df_factor_cs.reindex(df_count.index)['relacion c/s'], axis=0)

In [107]:
df_count_cs = df_count_cs.transpose()

In [108]:
df_count_cs.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,lg x screen,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g
0,0.0,0.0,0.056764,0.061601,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_count_cs['total_cs_factor'] = df_count_cs.iloc[:, 1:].sum(1)
df_count_cs.head()

,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,samsung galaxy tab 4 10.1 wi-fi + 3g,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g,total_cs_factor
0,0.0,0.0,0.056764,0.061601,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.217994
1,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [110]:
df_count_cs['person'] = df_model_viewed['person']

In [111]:
df_factorcs_by_model = df_count_cs['total_cs_factor'].to_frame()
df_factorcs_by_model.sort_values(by='total_cs_factor',ascending=False).head()

,total_cs_factor
1077,185.163478
9932,95.727989
8130,78.000000
5914,77.013552
6722,76.809802


In [112]:
df_count_cs['person'] = df_model_viewed['person']
df_count_cs.head()


,iphone 5s,samsung galaxy j5,iphone 6,iphone 6s,motorola moto g2 3g dual,samsung galaxy j7 prime,samsung galaxy s7,motorola moto g4 plus,samsung galaxy s6 flat,samsung galaxy s8,samsung galaxy s7 edge,...,samsung galaxy s3 slim duos,quantum go 4g,ipad mini wi-fi,sony xperia z5 premium,samsung galaxy gran neo plus duos,asus zenfone 3 max 32 gb,samsung galaxy s3 duos,samsung galaxy tab s 8.4 wi-fi + 4g,ipad air wi-fi + 4g,total_cs_factor,person
0,0.0,0.0,0.056764,0.061601,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.217994,000c79fe
1,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,000e619d
2,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,001001be
3,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,001802e4
4,0.0,0.0,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0019e639


In [113]:
df_count_cs['total_cs_factor'] = df_count_cs.iloc[:, 0:151].sum(1)

In [114]:
df_count_cs = df_count_cs [['person' , 'total_cs_factor']]
df_count_cs.head()

,person,total_cs_factor
0,000c79fe,0.435989
1,000e619d,0.000000
2,001001be,0.000000
3,001802e4,0.000000
4,0019e639,0.000000


In [115]:
features = df_count_cs.merge(df_count_cv, on='person' , how='right')

In [116]:
features.head()

,person,total_cs_factor,total_cv_factor
0,000c79fe,0.435989,0.029996
1,000e619d,0.000000,0.332996
2,001001be,0.000000,0.322710
3,001802e4,0.000000,0.043728
4,0019e639,0.000000,2.632312


In [117]:
features = features.merge(persons, on='person', how='right')
features =features.fillna(0)
features.shape


(38829, 3)

In [129]:
date_features = pd.read_csv('date.csv')
newr_devt_features = pd.read_csv('newr_devt.csv')

print(date_features.shape)
print(newr_devt_features.shape)

(38829, 7)
(38829, 6)


In [130]:
features = features.merge(date_features, on='person', how='left')
features = features.merge(newr_devt_features, on='person', how='left')
features.head()

,person,total_cs_factor,total_cv_factor,Unnamed: 0_x,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),Unnamed: 0_y,device_type,computer_smartphone,new_vs_returning,newr
0,000c79fe,0.435989,0.029996,3967,0.0,29.0,29.0,1.0,1.0,25377,0.0,1.0,0.0,1.0
1,000e619d,0.000000,0.332996,19254,2.0,18.0,15.0,4.0,1.0,10681,1.0,5.0,1.0,4.0
2,001001be,0.000000,0.322710,4729,5.0,22.0,16.0,1.0,2.0,13319,0.0,3.0,1.0,2.0
3,001802e4,0.000000,0.043728,6559,0.0,31.0,31.0,3.0,3.0,25369,0.0,1.0,0.0,1.0
4,0019e639,0.000000,2.632312,11150,20.0,22.0,2.0,1.0,2.0,1919,1.0,19.0,1.0,18.0


In [132]:
features = features.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1)
features.head()

KeyError: "['Unnamed: 0_x' 'Unnamed: 0_y'] not found in axis"

In [133]:
features.head()

,person,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
0,000c79fe,0.435989,0.029996,0.0,29.0,29.0,1.0,1.0,0.0,1.0,0.0,1.0
1,000e619d,0.000000,0.332996,2.0,18.0,15.0,4.0,1.0,1.0,5.0,1.0,4.0
2,001001be,0.000000,0.322710,5.0,22.0,16.0,1.0,2.0,0.0,3.0,1.0,2.0
3,001802e4,0.000000,0.043728,0.0,31.0,31.0,3.0,3.0,0.0,1.0,0.0,1.0
4,0019e639,0.000000,2.632312,20.0,22.0,2.0,1.0,2.0,1.0,19.0,1.0,18.0


## XGboost entrenamiento

In [134]:
df_train = df_labels.merge(features , left_on='person', right_on='person' , how='inner')

In [135]:
df_train.head()

,person,label,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
0,0566e9c1,0,0.0,0.232934,8.0,31.0,22.0,3.0,1.0,0.0,17.0,1.0,16.0
1,6ec7ee77,0,0.0,0.000000,0.0,28.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0
2,abe7a2fb,0,0.0,0.331879,27.0,29.0,2.0,1.0,2.0,0.0,22.0,1.0,21.0
3,34728364,0,0.0,0.330539,8.0,27.0,18.0,6.0,4.0,0.0,4.0,1.0,3.0
4,87ed62de,0,0.0,0.106222,0.0,19.0,18.0,5.0,4.0,1.0,1.0,0.0,1.0


In [136]:
df_train2 = pd.concat([df_train.loc[df_train['label'] == 1] , df_train.loc[df_train['label']==0].sample(7000)])

In [137]:
df_train2.head()

,person,label,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
5,db2c4d27,1,0.0,1.101025,0.0,2.0,2.0,2.0,2.0,0.0,107.0,1.0,113.0
25,8123457d,1,0.0,0.091337,0.0,30.0,30.0,2.0,2.0,0.0,3.0,1.0,2.0
46,e4b02ea2,1,0.0,0.197133,0.0,24.0,24.0,3.0,3.0,1.0,2.0,0.0,1.0
48,d8001b23,1,0.0,0.976650,24.0,25.0,1.0,4.0,1.0,1.0,14.0,1.0,18.0
54,7a472832,1,0.0,1.219481,21.0,31.0,9.0,3.0,2.0,1.0,8.0,1.0,7.0


In [138]:
df_train2.shape

(7980, 13)

Los labels me dan mi set para entrenar, los que no se encuentran en labels tengo que predecirlos

Si ven aca, de la columna label en adelante tenemos los features.

In [139]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
0,0.0,0.232934,8.0,31.0,22.0,3.0,1.0,0.0,17.0,1.0,16.0
1,0.0,0.000000,0.0,28.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.331879,27.0,29.0,2.0,1.0,2.0,0.0,22.0,1.0,21.0
3,0.0,0.330539,8.0,27.0,18.0,6.0,4.0,0.0,4.0,1.0,3.0
4,0.0,0.106222,0.0,19.0,18.0,5.0,4.0,1.0,1.0,0.0,1.0


## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [140]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)



Este es el arbol con sus hiperparametros

In [141]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [142]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [143]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.7455464766042895

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

In [144]:
X.isnull().sum()

total_cs_factor             0
total_cv_factor             0
diferencia 5                0
DAY(last month 5)           0
DAY(first month 5)          0
WEEKDAY(last month 5)       0
WEEKDAY(first month 5)      0
device_type               315
computer_smartphone       315
new_vs_returning          315
newr                      315
dtype: int64

In [145]:
X_predict = persons_to_predict.merge(features, on='person', how='left')
X_predict.head()


,person,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
0,4886f805,0.000000,0.048309,0.0,18.0,18.0,4.0,4.0,0.0,1.0,0.0,1.0
1,0297fc1e,0.000000,4.260824,26.0,28.0,2.0,0.0,2.0,0.0,95.0,1.0,94.0
2,2d681dd8,0.000000,0.152395,9.0,27.0,18.0,6.0,4.0,1.0,2.0,1.0,1.0
3,cccea85e,0.000000,9.629172,23.0,31.0,7.0,3.0,0.0,1.0,22.0,1.0,21.0
4,4c8a8b93,0.837551,1.741019,4.0,22.0,18.0,1.0,4.0,0.0,20.0,1.0,19.0


In [146]:
df_entrie = persons_to_predict
df_entrie['label'] = model.predict_proba(X_predict.iloc[:,1:])[:,1]
df_entrie.head()

,person,label
0,4886f805,0.184170
2,0297fc1e,0.238725
3,2d681dd8,0.223162
4,cccea85e,0.250028
5,4c8a8b93,0.224199


In [152]:
df_entrie['label'].value_counts()

0.184170    1994
0.189496    1212
0.200169    1211
0.187247    1010
0.196344     774
0.191013     450
0.207017     430
0.210915     413
0.188670     352
0.192383     330
0.185456     286
0.205178     278
0.188812     271
0.187871     252
0.198544     240
0.204558     233
0.193997     206
0.200844     201
0.234922     196
0.197631     185
0.197710     181
0.190783     171
0.238241     164
0.311301     160
0.191479     153
0.230420     128
0.197708     125
0.197233     123
0.208888     122
0.198251     112
            ... 
0.206746       1
0.366588       1
0.223108       1
0.245734       1
0.320180       1
0.222593       1
0.257746       1
0.222638       1
0.222940       1
0.289627       1
0.207248       1
0.246306       1
0.222856       1
0.258206       1
0.289406       1
0.289398       1
0.367467       1
0.230594       1
0.383038       1
0.222704       1
0.336024       1
0.398499       1
0.238306       1
0.207048       1
0.273467       1
0.367213       1
0.320323       1
0.320319      

In [1]:
features.head()

NameError: name 'features' is not defined